# Food Sales Prediction
- Zach Hanson
- Last updated 12/22/2022
 

## Project Description

### Task

### Data Dictionary

**Variable Name** | **Description**
--- | ---
Item_Identifier | Unique product ID
Item_Weight | Weight of product
Item_Fat_Content | Whether the product is low fat or regular
Item_Visibility | The percentage of total display area of all products in a store allocated to the particular product
Item_Type | The category to which the product belongs
Item_MRP | Maximum Retail Price (list price) of the product
Outlet_Identifier | Unique store ID
Outlet_Establishment_Year | The year in which store was established
Outlet_Size | The size of the store in terms of ground area covered
Outlet_Location_Type | The type of area in which the store is located
Outlet_Type | Whether the outlet is a grocery store or some sort of supermarket
Item_Outlet_Sales | Sales of the product in the particular store

### Import Libraries

In [1]:
#Pandas
import pandas as pd

#NumPy
import numpy as np

#Matplotlib
import matplotlib.pyplot as plt

#Seaborn
import seaborn as sns

#Imputers
from sklearn.impute import SimpleImputer

#Transformers
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

#Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_selector
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

#Models
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier

#Regression Metrics
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

#Setting global SciKit-Learn configuration
from sklearn import set_config
set_config(display='diagram')

### Functions

In [2]:
#Function to take the true and predicted values 
#and print MAE, MSE, RMSE, and R2 metrics
def evaluation_model(pipe,
                     model_name = '',
                     x_train = 'X_train', 
                     x_test ='X_test'):
    
    #Train
    mae = round(mean_absolute_error(y_train, pipe.predict(x_train)),4)
    mse = round(mean_squared_error(y_train, pipe.predict(x_train)),4)
    rmse = round(np.sqrt(mean_squared_error(y_train, pipe.predict(x_train))),4)
    r2 = round(r2_score(y_train, pipe.predict(x_train)),7)
    print(f'{model_name} Train Scores')
    print(f'MAE: {mae:,.4f} \nMSE: {mse:,.4f} \nRMSE: {rmse:,.4f} \nR2:{r2:.4f}\n')
    
    #Test
    mae = round(mean_absolute_error(y_test, pipe.predict(x_test)),4)
    mse = round(mean_squared_error(y_test, pipe.predict(x_test)),4)
    rmse = round(np.sqrt(mean_squared_error(y_test, pipe.predict(x_test))),4)
    r2 = round(r2_score(y_test, pipe.predict(x_test)),7)

    # Display the metrics for the model
    print(f'{model_name} Test Scores')
    print(f'MAE: {mae:,.4f} \nMSE: {mse:,.4f} \nRMSE: {rmse:,.4f} \nR2: {r2:.4f}\n')

## Loading and Inspecting Data

### Loading Data

In [3]:
#Import data
df_original = pd.read_csv('sales_predictions.csv')

### Inspect Data

In [4]:
df_original.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


- Data seems to be loaded properly

In [5]:
#Showing shape of data frame
df_original.shape

(8523, 12)

- There are 8,523 data entries
- There are 12 columns
 - 11 features and 1 target

In [6]:
#Viewing column names, data types, and count of non-null values
df_original.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


In [7]:
#Showing descriptive statistics for numerical data
df_original.describe(include='number')

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales
count,7060.000000,8523.000000,8523.000000,8523.000000,8523.000000
mean,12.857645,0.066132,140.992782,1997.831867,2181.288914
std,4.643456,0.051598,62.275067,8.371760,1706.499616
min,4.555000,0.000000,31.290000,1985.000000,33.290000
25%,8.773750,0.026989,93.826500,1987.000000,834.247400
50%,12.600000,0.053931,143.012800,1999.000000,1794.331000
75%,16.850000,0.094585,185.643700,2004.000000,3101.296400
max,21.350000,0.328391,266.888400,2009.000000,13086.964800


In [8]:
#Showing descriptive statistics for categorical data
df_original.describe(include='object')

,Item_Identifier,Item_Fat_Content,Item_Type,Outlet_Identifier,Outlet_Size,Outlet_Location_Type,Outlet_Type
count,8523,8523,8523,8523,6113,8523,8523
unique,1559,5,16,10,3,3,4
top,FDW13,Low Fat,Fruits and Vegetables,OUT027,Medium,Tier 3,Supermarket Type1
freq,10,5089,1232,935,2793,3350,5577


## Cleaning Data

### Remove Unnecessary Columns


In [9]:
df_original['Item_Identifier'].value_counts()

FDW13    10
FDG33    10
NCY18     9
FDD38     9
DRE49     9
         ..
FDY43     1
FDQ60     1
FDO33     1
DRF48     1
FDC23     1
Name: Item_Identifier, Length: 1559, dtype: int64

In [10]:
#Dropping Item Identifier column
df_original = df_original.drop(columns='Item_Identifier')

- Chose to remove this column because almost 20% of the data has a different value
- Most likely not crucial in predicting an items outlet sales, as we would be using these models to predict new items

### Remove Unnecessary Rows

#### Duplicates

In [11]:
#Checking for duplicate rows
dup = df_original.duplicated().sum()

print(f'There are {dup} duplicated rows.')

There are 0 duplicated rows.


- No duplicated rows

### Inspect Column Data-types for Errors

In [12]:
#Display column names and datatypes
df_original.dtypes

Item_Weight                  float64
Item_Fat_Content              object
Item_Visibility              float64
Item_Type                     object
Item_MRP                     float64
Outlet_Identifier             object
Outlet_Establishment_Year      int64
Outlet_Size                   object
Outlet_Location_Type          object
Outlet_Type                   object
Item_Outlet_Sales            float64
dtype: object

- All columns appear to have the correct data types

### Inspect Column Names for Errors

In [13]:
#Display column names
df_original.columns

Index(['Item_Weight', 'Item_Fat_Content', 'Item_Visibility', 'Item_Type',
       'Item_MRP', 'Outlet_Identifier', 'Outlet_Establishment_Year',
       'Outlet_Size', 'Outlet_Location_Type', 'Outlet_Type',
       'Item_Outlet_Sales'],
      dtype='object')

- No errors in column names noticeable

### Inspect Values for Errors

#### Numeric Columns

In [14]:
#Display summary statistics for numeric columns
df_original.describe(include='number')

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales
count,7060.000000,8523.000000,8523.000000,8523.000000,8523.000000
mean,12.857645,0.066132,140.992782,1997.831867,2181.288914
std,4.643456,0.051598,62.275067,8.371760,1706.499616
min,4.555000,0.000000,31.290000,1985.000000,33.290000
25%,8.773750,0.026989,93.826500,1987.000000,834.247400
50%,12.600000,0.053931,143.012800,1999.000000,1794.331000
75%,16.850000,0.094585,185.643700,2004.000000,3101.296400
max,21.350000,0.328391,266.888400,2009.000000,13086.964800


- No unusual values noted

#### Categorical Columns

In [15]:
#Checking Item_Fat_Content for inconsistencies
df_original['Item_Fat_Content'].value_counts()

Low Fat    5089
Regular    2889
LF          316
reg         117
low fat     112
Name: Item_Fat_Content, dtype: int64

- We can see we have 'LF', 'reg', and 'low fat'

In [16]:
#Updating so everything is consistent
df_original = df_original.replace(['LF', 'low fat', 'reg'],
                                  ['Low Fat', 'Low Fat', 'Regular'])

In [17]:
#Checking to make sure everything got replaced
df_original['Item_Fat_Content'].value_counts()

Low Fat    5517
Regular    3006
Name: Item_Fat_Content, dtype: int64

- Updated correctly

### Missing Values

In [18]:
#Display count of missing values by column
print(df_original.isna().sum())

Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64


- We can see there is 1,463 values missing in 'Item_Weight' column, and 2,410 values missing in 'Outlet_Size' column

In [19]:
#Showing percentage of missing values by column
print(df_original.isna().sum()/len(df_original)*100)

Item_Weight                  17.165317
Item_Fat_Content              0.000000
Item_Visibility               0.000000
Item_Type                     0.000000
Item_MRP                      0.000000
Outlet_Identifier             0.000000
Outlet_Establishment_Year     0.000000
Outlet_Size                  28.276428
Outlet_Location_Type          0.000000
Outlet_Type                   0.000000
Item_Outlet_Sales             0.000000
dtype: float64


In [20]:
#Showing total number of missing values
print(f'There are {df_original.isna().sum().sum()} total missing values.')

There are 3873 total missing values.


- We will need to impute both the Item_Weight column and Outlet_Size column
- Item_Weight we can impute with the mean of the other values recorded
- Outlet_Size we can impute with "Missing", since we would be adding a lot of bias with the large amount of missing data in this column

## Visualization Models

## Splitting Data

### Copying Data

In [21]:
#Copying original data frame to prevent data leakage
reg_df = df_original.copy()

In [22]:
#Checking to see if it got copied
reg_df.head()

,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


- Appears to have copied correctly

### Splitting Data

In [23]:
#Define features (X) and target (y)
target = 'Item_Outlet_Sales'
y = reg_df[target].copy()
X = reg_df.drop(columns = [target]).copy()

In [24]:
#Split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
#Random state 42 for consistency

## Preparing Data

### Column Selectors

In [25]:
#Numerical column selector
num_selector = make_column_selector(dtype_include='number')

#Categorical column selector
cat_selector = make_column_selector(dtype_include='object')

### Imputers

In [26]:
#Mean imputer for numerical data
mean_imputer = SimpleImputer(strategy='mean')

In [27]:
#Categorical imputer to fill missing nominal data with 'Missing'
missing_imputer = SimpleImputer(strategy='constant',   #Replace all missing values with same thing
                                fill_value='Missing')  #Replace all missing values with "Missing"

### Transformers

In [28]:
#Instantiate scaler and one hot encoder
scaler = StandardScaler()
ohe = OneHotEncoder(sparse=False, handle_unknown='ignore')

### Imputer Pipeline

#### Numerical Pipeline

In [29]:
num_pipeline = make_pipeline(mean_imputer, scaler)
num_pipeline

Pipeline(steps=[('simpleimputer', SimpleImputer()),
                ('standardscaler', StandardScaler())])

#### Categorical Pipeline

In [30]:
cat_pipeline = make_pipeline(missing_imputer, ohe)
#cat_pipeline = make_pipeline(ohe)

cat_pipeline

Pipeline(steps=[('simpleimputer',
                 SimpleImputer(fill_value='Missing', strategy='constant')),
                ('onehotencoder',
                 OneHotEncoder(handle_unknown='ignore', sparse=False))])

### Tuples

In [31]:
#Numerical Tuple
num_tuple = (num_pipeline, num_selector)

#Categorical Tuple
cat_tuple = (cat_pipeline, cat_selector)

### Column Transformer

In [32]:
preprocessor = make_column_transformer(num_tuple,
                                       cat_tuple,
                                       remainder='drop')
preprocessor

ColumnTransformer(transformers=[('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer()),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x000002E0E9D083A0>),
                                ('pipeline-2',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(fill_value='Missing',
                                                                strategy='constant')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False))]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x000002E0E9D1CD90>)])

## Regression Metrics

### Linear Regression

In [33]:
#Creating instance of model
lin_reg = LinearRegression()

In [34]:
#Create a model pipeline
lin_reg_pipeline = make_pipeline(preprocessor, lin_reg)

In [35]:
#Fit the model to training data
lin_reg_pipeline.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer()),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000002E0E9D083A0>),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(fill_value='Missing',
                                                                                 strategy='constant')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000002E0E9D1CD90>)])),
                ('linearregression', LinearRegression())])

In [36]:
#Running regression metrics on this model
evaluation_model(pipe = lin_reg_pipeline,
                 model_name = 'Linear Regression Model',
                 x_train = X_train,
                 x_test = X_test)

Linear Regression Model Train Scores
MAE: 847.1559 
MSE: 1,297,567.4886 
RMSE: 1,139.1082 
R2:0.5616

Linear Regression Model Test Scores
MAE: 804.1395 
MSE: 1,194,325.6467 
RMSE: 1,092.8521 
R2: 0.5671



- The R^2 value for this model is 0.5671 on our testing set.
- This model can use all the features to account for about 57% of the variance in the target value
- This model can predict the target value with about 1,100 sales error

### Regression Tree Model

In [37]:
#Generating Forest
rf = RandomForestRegressor(random_state=42)
#random state 42 for consistency

In [38]:
#Creating random forest pipeline
rf_pipe = make_pipeline(preprocessor, rf)

In [39]:
#Fitting to training data
rf_pipe.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer()),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000002E0E9D083A0>),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(fill_value='Missing',
                                                                                 strategy='constant')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000002E0E9D1CD90>)])),
                ('randomforestregressor',
                 RandomForestRegressor(random_state=42))])

In [40]:
#Finding max depth if we let this model have unlimited max_depth
est_depths = [estimator.get_depth() for estimator in rf.estimators_]
max(est_depths)

42

- If we let this model have an unlimited max_depth, the highest it will go is 42

In [41]:
#Finding best depth for this model 
depths = range(1, 42) #Max range found from above line

#Making new data frame to save information
scores = pd.DataFrame(index=depths, columns=['Test Score'])

for depth in depths:    
    model = RandomForestRegressor(max_depth=depth, random_state=42)
    model_pipe = make_pipeline(preprocessor, model)
    model_pipe.fit(X_train, y_train)
    
    scores.loc[depth, 'Train Score'] = model_pipe.score(X_train, y_train)
    scores.loc[depth, 'Test Score'] = model_pipe.score(X_test, y_test)


In [42]:
#Sorting scores to see which depth gives us best R^2 value
sorted_scores = scores.sort_values(by='Test Score',
                                   ascending=False)
sorted_scores.head()

,Test Score,Train Score
5,0.602609,0.610541
6,0.60128,0.624538
7,0.598417,0.643105
4,0.596632,0.595925
8,0.596497,0.666334


- Depth of 5 yields the highest R^2 value for our testing set

In [43]:
#Finding best n_estimators using our best depth
n_est = [50, 100, 150, 200, 250, 300]
scores2 = pd.DataFrame(index=n_est, columns=['Test Score', 'Train Score'])

for n in n_est:
    model = RandomForestRegressor(max_depth=5, n_estimators=n, random_state=42)
    model_pipe = make_pipeline(preprocessor, model)
    model_pipe.fit(X_train, y_train)
    
    scores2.loc[n, 'Train Score'] = model_pipe.score(X_train, y_train)
    scores2.loc[n, 'Test Score'] = model_pipe.score(X_test, y_test)

In [44]:
#Sorting scores to see which n_estimator gives us best R^2 value
sorted_scores2 = scores2.sort_values(by='Test Score',
                                     ascending=False)
sorted_scores2.head()

,Test Score,Train Score
150,0.602977,0.610368
250,0.602932,0.610488
300,0.60291,0.610445
200,0.602878,0.610382
100,0.602609,0.610541


- n_estimator of 150 yields the highest R^2 value for our testing set

In [64]:
#Generating optimal random forest
rf_opt = RandomForestRegressor(max_depth=5,
                               n_estimators=150,
                               random_state=42)

In [65]:
#Creating pipeline
rf_opt_pipe = make_pipeline(preprocessor, rf_opt)

In [66]:
rf_opt_pipe.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer()),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000002E0E9D083A0>),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(fill_value='Missing',
                                                                                 strategy='constant')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000002E0E9D1CD90>)])),
                ('randomforestregressor',
                 RandomForestRegressor(max_depth=5, n_estimators=150,
                                       random_state=42))])

In [67]:
#Running regression metrics on this model
evaluation_model(pipe = rf_opt_pipe,
                 model_name = 'Optimal Random Forest Model',
                 x_train = X_train,
                 x_test = X_test)

Optimal Random Forest Model Train Scores
MAE: 755.4558 
MSE: 1,153,098.6945 
RMSE: 1,073.8243 
R2:0.6104

Optimal Random Forest Model Test Scores
MAE: 728.2434 
MSE: 1,095,378.3181 
RMSE: 1,046.6032 
R2: 0.6030



- The R^2 value for this model is 0.6063 on our testing set.
- This model can use all the features to account for about 60% of the variance in the target value
- This model can predict the target value with about 1,050 sales error

### Recommendation